In [1]:
import sqlite3
import pandas as pd

DB_PATH = r"C:\Users\nvbma\my_testdb.db"
OUTPUT_FILE = "output_pandas.csv"


def validate_tables(conn):
    required_tables = {"Customer", "Sales", "Orders", "Items"}
    existing = pd.read_sql(
        "SELECT name FROM sqlite_master WHERE type='table';", conn
    )["name"].tolist()

    missing = required_tables - set(existing)
    if missing:
        raise Exception(f"Missing tables: {missing}")


def main():
    conn = None
    try:
        conn = sqlite3.connect(DB_PATH)

        # Validate required tables exist
        validate_tables(conn)

        # Read tables
        customers = pd.read_sql("SELECT * FROM Customer", conn)
        sales = pd.read_sql("SELECT * FROM Sales", conn)
        orders = pd.read_sql("SELECT * FROM Orders", conn)
        items = pd.read_sql("SELECT * FROM Items", conn)

        # Merge tables (Header → Detail relationship)
        df = (
            customers
            .merge(sales, on="customer_id", how="inner")
            .merge(orders, on="sales_id", how="inner")
            .merge(items, on="item_id", how="inner")
        )

        # Ensure quantity is numeric
        df["quantity"] = pd.to_numeric(df["quantity"], errors="coerce")

        # Apply business rules
        df = df[
            (df["age"].between(18, 35)) &        # Age filter
            (df["quantity"].notna())             # Ignore NULL purchases
        ]

        # Group and aggregate safely
        result = (
            df.groupby(["customer_id", "age", "item_name"], as_index=False)["quantity"]
              .sum()
        )

        # Remove zero totals
        result = result[result["quantity"] > 0]

        # Rename columns for output
        result.rename(columns={
            "customer_id": "Customer",
            "age": "Age",
            "item_name": "Item",
            "quantity": "Quantity"
        }, inplace=True)

        # Export
        result.to_csv(OUTPUT_FILE, sep=";", index=False)

        print("Pandas solution completed successfully.")

    except Exception as e:
        print("Error:", e)

    finally:
        if conn:
            conn.close()


if __name__ == "__main__":
    main()

Pandas solution completed successfully.
